In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import pickle


C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.


In [2]:
trainDF =pd.read_csv('data_ful_cleard.csv',encoding='utf-8',delimiter=';',error_bad_lines=False)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)



In [4]:
list(encoder.classes_)

['egyptian', 'general', 'gulf', 'iraqi', 'levantine', 'maghrebi', 'tunisien']

In [17]:
list(encoder.inverse_transform([0,2, 2, 1]))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['egyptian', 'gulf', 'gulf', 'general']

In [15]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [4]:
pickle.dump(count_vect.vocabulary_,open("count_vect.pkl","wb"))


In [7]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("count_vect.pkl", "rb")))

In [6]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return classifier,metrics.accuracy_score(predictions, valid_y)

In [15]:
# Naive Bayes on Count Vectors
model,accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)



filename = 'NB_Count_Vectors.pkl'

pickle.dump(model, open(filename, 'wb'))


# # Naive Bayes on Word Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("NB, WordLevel TF-IDF: ", accuracy)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("NB, N-Gram Vectors: ", accuracy)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.7295475081670953


In [16]:
loaded_model = pickle.load(open("NB_Count_Vectors.pkl", 'rb'))
# loaded_model = joblib.load("NB_Count_Vectors.sav")

encoder.inverse_transform(loaded_model.predict(loaded_vec.transform(["هوا في ايه"])))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['egyptian'], dtype=object)

###  TFIDF

In [3]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [4]:
pickle.dump(tfidf_vect.vocabulary_,open("Tfidf_word_vect.pkl","wb"))

In [7]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("Tfidf_word_vect.pkl", "rb")))

In [8]:
model,accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

NB, WordLevel TF-IDF:  0.5630777785500799


In [9]:

pickle.dump(model, open("NB_WordLevel_TF_IDF.pkl", 'wb'))

In [10]:
loaded_model = pickle.load(open("NB_WordLevel_TF_IDF.pkl", 'rb'))
# loaded_model = joblib.load("NB_Count_Vectors.sav")

encoder.inverse_transform(loaded_model.predict(loaded_vec.transform(["هوا في ايه"])))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['egyptian'], dtype=object)

In [11]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

pickle.dump(tfidf_vect.vocabulary_,open("tfidf_vect_ngram_chars.pkl","wb"))



In [4]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])


pickle.dump(tfidf_vect_ngram.vocabulary_,open("tfidf_vect_ngram.pkl","wb"))

In [13]:


model,accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)


pickle.dump(model, open("NB_CharLevel_Vectors.pkl", 'wb'))


NB, CharLevel Vectors:  0.6202474456106207


In [14]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("tfidf_vect_ngram_chars.pkl", "rb")))
loaded_model = pickle.load(open("NB_CharLevel_Vectors.pkl", 'rb'))

encoder.inverse_transform(loaded_model.predict(loaded_vec.transform(["هوا في ايه"])))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['egyptian'], dtype=object)

In [16]:
model,accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

pickle.dump(model, open("LR_Count_Vectors.pkl", 'wb'))

LR, Count Vectors:  0.7040731215680823


In [18]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("count_vect.pkl", "rb")))
loaded_model = pickle.load(open("LR_Count_Vectors.pkl", 'rb'))

encoder.inverse_transform(loaded_model.predict(loaded_vec.transform(["هوا في ايه"])))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['egyptian'], dtype=object)

In [19]:
model,accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)


pickle.dump(model, open("LR_CharLevel_Vectors.pkl", 'wb'))

LR, CharLevel Vectors:  0.670153610898728


In [20]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("tfidf_vect_ngram_chars.pkl", "rb")))
loaded_model = pickle.load(open("LR_CharLevel_Vectors.pkl", 'rb'))

encoder.inverse_transform(loaded_model.predict(loaded_vec.transform(["هوا في ايه"])))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['general'], dtype=object)

In [21]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

pickle.dump(model, open("SVM_N_Gram_Vectors.pkl", 'wb'))


SVM, N-Gram Vectors:  (SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), 0.21297004239938833)


In [ ]:
pickle.dump(tfidf_vect.vocabulary_,open("tfidf_vect_ngram.pkl","wb"))

In [ ]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("tfidf_vect_ngram.pkl", "rb")))
loaded_model = pickle.load(open("SVM_N_Gram_Vectors.pkl", 'rb'))

encoder.inverse_transform(loaded_model.predict(loaded_vec.transform(["هوا في ايه"])))

In [13]:
from keras.preprocessing import text, sequence

token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])

pickle.dump(token, open('fit_on_texts.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

tokenizer = pickle.load(open('fit_on_texts.pkl', 'rb'))

valid_seq_x = sequence.pad_sequences(tokenizer.texts_to_sequences(valid_x), maxlen=70)

valid_seq_x

array([[     0,      0,      0, ...,   9530,   1351,     16],
       [     0,      0,      0, ...,   1684,    115, 203332],
       [     0,      0,      0, ...,  12316,    652,     16],
       ...,
       [     0,      0,      0, ...,      4,  13746,   1101],
       [     0,      0,      0, ...,  92085, 120614, 213357],
       [     0,      0,      0, ...,    120,  40837, 158368]])

In [ ]:
from keras.models import load_model

model = load_model('NN_NgramLevelTFIDFVectors.h5')
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)


In [9]:
from keras.models import load_model

loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("tfidf_vect_ngram_chars.pkl", "rb")))
loaded_model = load_model('models/NN_NgramLevelTFIDFVectors.h5')
predictions = loaded_model.predict(loaded_vec.transform(["هوا في ايه"]))
predictions = predictions.argmax(axis=-1)
encoder.inverse_transform(predictions)

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['egyptian'], dtype=object)

In [14]:
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("count_vect.pkl", "rb")))
loaded_model = pickle.load(open("Xgb1_CountVectors.pkl", 'rb'))

encoder.inverse_transform(loaded_model.predict((loaded_vec.transform(["هوا في ايه"])).tocsc()))

C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\youne\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['egyptian'], dtype=object)